In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24546
2,Huelva,Confirmados PDIA 14 días,383
3,Huelva,Tasa PDIA 14 días,"74,63413683574643"
4,Huelva,Confirmados PDIA 7 días,119
5,Huelva,Total Confirmados,24768
6,Huelva,Curados,16008
7,Huelva,Fallecidos,298


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24546.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6546.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5137 personas en los últimos 7 días 

Un positivo PCR cada 1653 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24546.0,119.0,383.0,513170.0,23.189197,74.634137,24.0
Condado-Campiña,6857.0,47.0,149.0,156231.0,30.083658,95.371597,19.0
Huelva-Costa,14971.0,70.0,218.0,289548.0,24.175612,75.289762,9.0
Lucena del Puerto,151.0,5.0,5.0,3261.0,153.327200,153.327200,6.0
Moguer,825.0,9.0,28.0,21867.0,41.157909,128.046829,4.0
Palma del Condado (La),974.0,8.0,14.0,10801.0,74.067216,129.617628,4.0
Huelva (capital),6546.0,28.0,87.0,143837.0,19.466479,60.485132,3.0
Ayamonte,1189.0,4.0,15.0,21104.0,18.953753,71.076573,3.0
Bollullos Par del Condado,608.0,3.0,23.0,14387.0,20.852158,159.866546,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,174.0,2.0,9.0,2885.0,69.324090,311.958406,0.0
Rosal de la Frontera,25.0,1.0,3.0,1697.0,58.927519,176.782557,0.0
Cartaya,939.0,14.0,33.0,20083.0,69.710701,164.318080,2.0
Bollullos Par del Condado,608.0,3.0,23.0,14387.0,20.852158,159.866546,2.0
Lucena del Puerto,151.0,5.0,5.0,3261.0,153.327200,153.327200,6.0
Cabezas Rubias,29.0,1.0,1.0,706.0,141.643059,141.643059,0.0
Almonte,854.0,13.0,32.0,24507.0,53.046068,130.574938,1.0
Palma del Condado (La),974.0,8.0,14.0,10801.0,74.067216,129.617628,4.0
Puebla de Guzmán,84.0,4.0,4.0,3092.0,129.366106,129.366106,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,309.0,1.0,9.0,7939.0,12.596045,113.364404,0.0,0.111111
Sierra de Huelva-Andévalo Central,2298.0,1.0,8.0,67391.0,1.483878,11.871021,0.0,0.125000
Bollullos Par del Condado,608.0,3.0,23.0,14387.0,20.852158,159.866546,2.0,0.130435
Palos de la Frontera,449.0,2.0,12.0,11742.0,17.032873,102.197241,1.0,0.166667
Lepe,1989.0,5.0,30.0,27880.0,17.934003,107.604017,0.0,0.166667
Punta Umbría,658.0,2.0,11.0,15355.0,13.025073,71.637903,1.0,0.181818
Trigueros,228.0,1.0,5.0,7862.0,12.719410,63.597049,0.0,0.200000
Villablanca,174.0,2.0,9.0,2885.0,69.324090,311.958406,0.0,0.222222
Gibraleón,510.0,1.0,4.0,12737.0,7.851142,31.404569,1.0,0.250000
